# Desenvolvimento de PLN (Processamento de Linguagem Natural)
> Felipe Seleme Ribeiro

## Classificação de Texto

> Para esta etapa, será utilizado um cenário fictício de uma empresa que recebe milhares de e-mails de clientes diariamente.  
> Os objetivos são:  
>  
> - Classificar os e-mails automaticamente em categorias, como "Suporte Técnico", "Consulta Comercial" e "Reclamação".
> - Gerar respostas automáticas baseadas na categoria detectada.

### Implementação de Classificação com BERT:

#### Preparando o ambiente:

É recomendável criar um ambiente virtual e instalar as bibliotecas necessárias:

```bash
# Criando o ambiente virtual
python3 -m venv venv
# Ativando o ambiente virtual
source venv/bin/activate  # No Windows, use venv\Scripts\activate

# Instalando as bibliotecas necessárias para esse projeto
pip install -r requirements.txt

In [104]:
# Importação das bibliotecas necessárias
import pandas as pd  # Biblioteca para manipulação de dados
from sklearn.model_selection import train_test_split  # Biblioteca para divisão de dados
from sklearn.preprocessing import LabelEncoder  # Biblioteca para codificação de dados
from transformers import BertTokenizer, BertForSequenceClassification  # Biblioteca para tokenização e classificação
from torch.optim import AdamW  # Biblioteca para otimização de pesos
import torch  # Biblioteca para manipulação de dados
import re  # Biblioteca para manipulação de expressões regulares
print('✅️ Bibliotecas importadas com sucesso!')

✅️ Bibliotecas importadas com sucesso!


Configurar o dispositivo para utilizar o processamento da GPU se compatível:

In [105]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('✅️ Dispositivo configurado como:', device)

✅️ Dispositivo configurado como: cuda


#### Obtenção dos Dados:

Definir um dataset de exemplo, contendo **e-mails** e **catagorias**:

In [106]:
# Dataset fictício em estrutura de dicionário Python (tipicamente encontrada em APIs através de arquivos JSON)
data = {
    "email": [
        "Preciso de ajuda com o sistema.",
        "Gostaria de informações sobre preços.",
        "Não estou satisfeito com o atendimento.",
        "O sistema está com problemas.",
        "Vocês oferecem planos de assinatura?"
    ],
    "categoria": ["Suporte Técnico", "Consulta Comercial", "Reclamação", "Suporte Técnico", "Consulta Comercial"]
}

# Criando o DataFrame com Pandas
data_df = pd.DataFrame(data)
print('✅️ DataFrame criado com sucesso.')
print("Dataframe dos e-mails de exemplo:")
print(data_df)

✅️ DataFrame criado com sucesso.
Dataframe dos e-mails de exemplo:
                                     email           categoria
0          Preciso de ajuda com o sistema.     Suporte Técnico
1    Gostaria de informações sobre preços.  Consulta Comercial
2  Não estou satisfeito com o atendimento.          Reclamação
3            O sistema está com problemas.     Suporte Técnico
4     Vocês oferecem planos de assinatura?  Consulta Comercial


#### Limpeza e Manipulação dos Dados:

In [107]:
# Função para limpar e normalizar o texto
def clean_text(text):
    """Limpa e normaliza o texto."""
    text = text.lower()  # Converte para minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # Remove pontuação
    return text

# Insere o texto normalizado no Dataframe
data_df['email_cleaned'] = data_df['email'].apply(clean_text)

print('✅️ DataFrame normalizado com sucesso.')
print("E-mails normalizados:")
print(data_df[['email', 'email_cleaned']])

✅️ DataFrame normalizado com sucesso.
E-mails normalizados:
                                     email  \
0          Preciso de ajuda com o sistema.   
1    Gostaria de informações sobre preços.   
2  Não estou satisfeito com o atendimento.   
3            O sistema está com problemas.   
4     Vocês oferecem planos de assinatura?   

                            email_cleaned  
0          preciso de ajuda com o sistema  
1    gostaria de informações sobre preços  
2  não estou satisfeito com o atendimento  
3            o sistema está com problemas  
4     vocês oferecem planos de assinatura  


#### Preparação do Modelo para o BERT:

In [108]:
# Codificação das categorias (target) - transforma as categorias de texto em números inteiros.
label_encoder = LabelEncoder()
data_df['label'] = label_encoder.fit_transform(data_df['categoria'])

# Tokenização BERT - quebra os textos em sub-palavras e adiciona tokens especiais necessários.
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Divisão dos dados em conjuntos de treinamento (80%) e teste (20%).
X_train, X_test, y_train, y_test = train_test_split(
    data_df['email_cleaned'],  # Os textos processados.
    data_df['label'],          # Os rótulos numéricos correspondentes.
    test_size=0.2,             # Proporção de dados para o conjunto de teste.
    random_state=42            # Para reprodutibilidade.
)

# Codificação - converte os textos e rótulos em formatos compatíveis com o modelo BERT.
def encode_data(texts, labels, tokenizer, max_length=128):
    inputs = tokenizer(
        texts.to_list(),              # Lista de textos para tokenizar.
        padding=True,                 # Adiciona preenchimento nas sequências menores.
        truncation=True,              # Trunca sequências maiores que `max_length`.
        max_length=max_length,        # Comprimento máximo permitido.
        return_tensors="pt"           # Retorna tensores PyTorch.
    )
    labels = torch.tensor(labels.to_list())  # Converte os rótulos em tensores PyTorch.
    return inputs, labels

# Aplicando a codificação aos dados de treinamento e teste.
train_inputs, train_labels = encode_data(X_train, y_train, tokenizer)
test_inputs, test_labels = encode_data(X_test, y_test, tokenizer)

# Inicialização do modelo BERT
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",  # Modelo BERT pré-treinado.
    num_labels=3          # Número de categorias de saída.
)
model.to(device)  # Move o modelo para o dispositivo (CPU ou GPU).
# Configuração do otimizador
optimizer = AdamW(model.parameters(), lr=5e-5)  # Taxa de aprendizado padrão recomendada para BERT.

print("✅️ Dados codificados para o modelo BERT.")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅️ Dados codificados para o modelo BERT.


#### Treinamento (simplificado - exemplo apenas com 2 epocas):

In [109]:
model.train()  # Coloca o modelo em modo de treinamento
for epoch in range(2):  # Loop de treinamento por 2 épocas
    inputs = {key: val.to(device) for key, val in train_inputs.items()}  # Move as entradas (inputs) para o dispositivo (CPU ou GPU)
    labels = train_labels.to(device)   # Move os rótulos (labels) para o dispositivo (GPU ou CPU)

    optimizer.zero_grad()   # Zera os gradientes acumulados das iterações anteriores
    outputs = model(**inputs, labels=labels)  # Passa as entradas pelo modelo para calcular as previsões
    loss = outputs.loss  # A perda é extraída da saída do modelo
    loss.backward()   # Calcula os gradientes em relação à perda
    optimizer.step()  # Atualiza os pesos do modelo

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

Epoch 1, Loss: 0.9944085478782654
Epoch 2, Loss: 1.0881786346435547


O valor da perda diminui da primeira para a segunda época. O que indica que o modelo está aprendendo.

#### Teste de Classificação do Modelo:

In [110]:
model.eval()  # Coloca o modelo em modo de avaliação

# Novo email que será classificado
new_email = "Preciso de suporte com um problema no sistema."

# Limpeza do texto para garantir que o novo email passe pelo mesmo pré-processamento que os dados de treinamento.
new_email_cleaned = clean_text(new_email)

# Tokeniza o novo email
new_input = tokenizer(new_email_cleaned, return_tensors="pt", padding=True, truncation=True).to(device)

# Desativa o cálculo de gradientes para acelerar a inferência e economizar memória.
with torch.no_grad():
    output = model(**new_input)  # Passa o texto tokenizado pelo modelo
    predicted = torch.argmax(output.logits, dim=1).item()  # Obtém a categoria prevista
    categoria = label_encoder.inverse_transform([predicted])[0]   # Converte a classe prevista de volta para seu valor original de categoria
    print(f"Categoria Prevista: {categoria}")

Categoria Prevista: Suporte Técnico


Sucesso! O modelo fez uma categorização correta do e-mail.

---

Manipulação de Dados

Considerando o seguinte conjunto de dados textuais:
- O gato está no telhado.
- A chuva cai sem parar.
- Gosto de assistir filmes nos finais de semana.
- Ele está estudando para as provas finais.

In [111]:
# Conjunto de dados textuais
texts = [
    "O gato está no telhado.",
    "A chuva cai sem parar.",
    "Gosto de assistir filmes nos finais de semana.",
    "Ele está estudando para as provas finais."
]
print('✅️ Textos carregados com sucesso.')

✅️ Textos carregados com sucesso.


### Realizar a limpeza dos textos, incluindo a remoção de stopwords, normalização (caixa baixa), e tokenização.

Para essa etapa, foi escolhida a biblioteca **NLTK**.  
Importando bibliotecas adicionais necessárias:

In [112]:
import nltk  # Biblioteca para processamento de linguagem natural
from nltk.corpus import stopwords  # Conjunto de stopwords (como "e", "o", "a", "de") que não são úteis em tarefas de análise de texto
from nltk.tokenize import word_tokenize  # Divide o texto em palavras ou tokens
from nltk.stem import WordNetLemmatizer  # Reduz as palavras às suas formas base (como "correndo" para "correr")
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer  # TF-IDF e Bag of Words (BoW) explicados mais abaixo.
print('✅️ Bibliotecas adicionais importadas com sucesso.')

✅️ Bibliotecas adicionais importadas com sucesso.


In [113]:
# Baixar pacotes necessários do NLTK
nltk.download('punkt')  # Baixa os pacotes necessários para tokenização.
nltk.download('stopwords')  # Baixa o pacote de stopwords.

# nltk.download('all', force=True)  # Baixa todos os pacotes. Descomente para garantir a execução.

[nltk_data] Downloading package punkt to
[nltk_data]     /home/felipeselemeribeiro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/felipeselemeribeiro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [114]:
# Remoção de stopwords
stop_words = set(stopwords.words('portuguese'))

# Normalização (caixa baixa) e Tokenização
def clean_text(text):
    tokens = word_tokenize(text.lower())  # Converte o texto para minúsculas e tokeniza.
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]  # Remove stopwords e palavras não alfanuméricas.
    return tokens
print('✅️ Stopwords removidos. Texto tokenizado.')

# Limpeza de cada texto na lista
norm_texts = [clean_text(text) for text in texts]

# Exibe os resultados
for original, cleaned in zip(texts, norm_texts):
    print("Texto original:", original)
    print("Texto limpo e tokenizado:", cleaned, "\n")

✅️ Stopwords removidos. Texto tokenizado.
Texto original: O gato está no telhado.
Texto limpo e tokenizado: ['gato', 'telhado'] 

Texto original: A chuva cai sem parar.
Texto limpo e tokenizado: ['chuva', 'cai', 'parar'] 

Texto original: Gosto de assistir filmes nos finais de semana.
Texto limpo e tokenizado: ['gosto', 'assistir', 'filmes', 'finais', 'semana'] 

Texto original: Ele está estudando para as provas finais.
Texto limpo e tokenizado: ['estudando', 'provas', 'finais'] 



Os stopwords foram removidos e o texto foi tokenizado com sucesso.

### Aplicando lematização aos textos.

Para essa etapa, utilizaremos a biblioteca **SpaCy**.  

In [115]:
import spacy  # Biblioteca SpaCy para processamento de linguagem natural
print("✅ Biblioteca SpaCy importada com sucesso.")

# Baixar o modelo de português do SpaCy
import os
if not os.path.exists("pt_core_news_sm"):
    os.system("python -m spacy download pt_core_news_sm")

# Carregar o modelo de português
nlp = spacy.load("pt_core_news_sm")

✅ Biblioteca SpaCy importada com sucesso.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 59.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [116]:
# Função para limpeza e lematização usando SpaCy
def clean_and_lemmatize_spacy(text):
    doc = nlp(text.lower())  # Processa o texto e converte para caixa baixa
    tokens = [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]  # Lematiza, remove stopwords e não alfanuméricos
    return tokens

# Aplica a lematização com SpaCy em cada texto
lemmatized_texts = [clean_and_lemmatize_spacy(text) for text in texts]

# Exibe os resultados
for original, cleaned in zip(texts, lemmatized_texts):
    print("Texto original:", original)
    print("Texto limpo e lematizado:", cleaned, "\n")


Texto original: O gato está no telhado.
Texto limpo e lematizado: ['gato', 'telhado'] 

Texto original: A chuva cai sem parar.
Texto limpo e lematizado: ['chuva', 'cair', 'parar'] 

Texto original: Gosto de assistir filmes nos finais de semana.
Texto limpo e lematizado: ['gostar', 'assistir', 'filme', 'final', 'semana'] 

Texto original: Ele está estudando para as provas finais.
Texto limpo e lematizado: ['estudar', 'prova', 'final'] 



### Criando vetores de característica utilizando TF-IDF e Bag of Words.

In [117]:
# Imprime os textos limpos, tokenizados e lematizados
print(lemmatized_texts)

[['gato', 'telhado'], ['chuva', 'cair', 'parar'], ['gostar', 'assistir', 'filme', 'final', 'semana'], ['estudar', 'prova', 'final']]


In [118]:
# Converter listas de tokens para strings
lemmatized_strings = [" ".join(tokens) for tokens in lemmatized_texts]
print(lemmatized_strings)

['gato telhado', 'chuva cair parar', 'gostar assistir filme final semana', 'estudar prova final']


TF-IDF - Cria vetores de características com base na frequência relativa e na importância das palavras.

In [119]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectors = tfidf_vectorizer.fit_transform(lemmatized_strings)

print("Matriz TF-IDF:")
print(tfidf_vectors.toarray())
print("Vocabulário TF-IDF:", tfidf_vectorizer.get_feature_names_out())

Matriz TF-IDF:
[[0.         0.         0.         0.         0.         0.
  0.70710678 0.         0.         0.         0.         0.70710678]
 [0.         0.57735027 0.57735027 0.         0.         0.
  0.         0.         0.57735027 0.         0.         0.        ]
 [0.46516193 0.         0.         0.         0.46516193 0.36673901
  0.         0.46516193 0.         0.         0.46516193 0.        ]
 [0.         0.         0.         0.61761437 0.         0.48693426
  0.         0.         0.         0.61761437 0.         0.        ]]
Vocabulário TF-IDF: ['assistir' 'cair' 'chuva' 'estudar' 'filme' 'final' 'gato' 'gostar'
 'parar' 'prova' 'semana' 'telhado']


Bag of Words (BoW) - Constrói vetores de características considerando apenas a contagem de palavras.

In [120]:
# Bag of Words (BoW)
bow_vectorizer = CountVectorizer()
bow_vectors = bow_vectorizer.fit_transform(lemmatized_strings)

print("\nMatriz Bag of Words:")
print(bow_vectors.toarray())
print("Vocabulário Bag of Words:", bow_vectorizer.get_feature_names_out())


Matriz Bag of Words:
[[0 0 0 0 0 0 1 0 0 0 0 1]
 [0 1 1 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 1 1 0 1 0 0 1 0]
 [0 0 0 1 0 1 0 0 0 1 0 0]]
Vocabulário Bag of Words: ['assistir' 'cair' 'chuva' 'estudar' 'filme' 'final' 'gato' 'gostar'
 'parar' 'prova' 'semana' 'telhado']
